1.	Write a series of for loops that compute the average value in torch_tensor3d (refer to torch_tensor3d in Week01 Activity)

In [ ]:
import torch

torch_tensor3d = torch.tensor([
[
[ 1, 2, 3],
[ 4, 5, 6],
],
[
[ 7, 8, 9],
[10, 11, 12],
],
[
[13, 14, 15],
[16, 17, 18],
],
[
[19, 20, 21],
[22, 23, 24],
]
])


In [ ]:
total = 0
count = 0

for i in range(torch_tensor3d.shape[0]):
    for j in range(torch_tensor3d.shape[1]):
        for k in range(torch_tensor3d.shape[2]):
            total += torch_tensor3d[i][j][k]
            count += 1

average = total / count
print("Average value (manual):", average)


In [ ]:
print(torch.mean(torch_tensor3d.float()))


	For every power of 2 (i.e., 2i or 2**i ) up to 211, create a random matrix X ∈ R  2i;2i (i.e., X.shape should give (2**i, 2**i)). Time how long it takes to compute XX (i.e., X @ X) on a CPU and on a GPU, and plot the speedup. For what matrix sizes is the CPU faster than the GPU?



In [ ]:
import torch
import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt  # For plotting axis scale and labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

cpu_times = []
gpu_times = []
sizes = []

for i in range(1, 12):
    n = 2 ** i
    sizes.append(n)
    X_cpu = torch.rand((n, n))

    # Time CPU
    start = time.time()
    _ = X_cpu @ X_cpu
    end = time.time()
    cpu_times.append(end - start)

    # Time GPU
    X_gpu = X_cpu.to(device)
    torch.cuda.synchronize()
    start = time.time()
    _ = X_gpu @ X_gpu
    torch.cuda.synchronize()
    end = time.time()
    gpu_times.append(end - start)

speedup = [c / g if g > 0 else 0 for c, g in zip(cpu_times, gpu_times)]

df = pd.DataFrame({"Matrix Size": sizes, "Speedup": speedup})

sns.lineplot(data=df, x="Matrix Size", y="Speedup", marker="o")
plt.xscale('log', base=2)
plt.xlabel("Matrix size (2^i)")
plt.ylabel("Speedup (CPU_time / GPU_time)")
plt.title("CPU vs GPU Speedup for Matrix Multiplication")
plt.grid(True)
plt.show()


In [ ]:
import torch
import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

cpu_times = []
gpu_times = []
sizes = []

# Measure matrix multiplication time on CPU and GPU
for i in range(1, 12):
    n = 2 ** i
    sizes.append(n)
    X_cpu = torch.rand((n, n))

    # CPU timing
    start = time.time()
    _ = X_cpu @ X_cpu
    end = time.time()
    cpu_times.append(end - start)

    # GPU timing
    X_gpu = X_cpu.to(device)
    torch.cuda.synchronize()
    start = time.time()
    _ = X_gpu @ X_gpu
    torch.cuda.synchronize()
    end = time.time()
    gpu_times.append(end - start)

# Compute speedup (CPU_time / GPU_time)
speedup = [c / g if g > 0 else 0 for c, g in zip(cpu_times, gpu_times)]

# --- Create two subplots ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: CPU vs GPU Time
axes[0].plot(sizes, cpu_times, marker='o', label='CPU Time')
axes[0].plot(sizes, gpu_times, marker='o', label='GPU Time')
axes[0].set_xscale('log', base=2)
axes[0].set_yscale('log')
axes[0].set_xlabel('Matrix Size (2^i)')
axes[0].set_ylabel('Time (s)')
axes[0].set_title('CPU vs GPU Time for Matrix Multiplication')
axes[0].grid(True)
axes[0].legend()

# Right: GPU Speedup
axes[1].plot(sizes, speedup, marker='o', color='purple', label='Speedup (CPU/GPU)')
axes[1].axhline(1, color='r', linestyle='--', label='CPU = GPU')
axes[1].set_xscale('log', base=2)
axes[1].set_xlabel('Matrix Size (2^i)')
axes[1].set_ylabel('Speedup')
axes[1].set_title('Speedup of GPU over CPU')
axes[1].grid(True)
axes[1].legend()

plt.tight_layout()
plt.show()


	We used PyTorch to find the numeric solution to f(x) = (x - 2)2. Write code that
finds the solution to f(x) = sin(x - 2) · (x + 2)2 +√(| cos(x)|). What answer do you get?


In [ ]:
import torch

def f(x):
    return torch.sin(x - 2) * (x + 2)**2 + torch.sqrt(torch.cos(x))

x_param = torch.nn.Parameter(torch.tensor([0.0]), requires_grad=True)
optimizer = torch.optim.SGD([x_param], lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = f(x_param)
    loss.backward()
    optimizer.step()

print("Optimized x:", x_param.data)
print("Function value at optimized x:", f(x_param).item())


4.	Write a new function that takes two inputs, x and y, where
f(x; y) = exp(sin(x)2) /(x - y)2 + (x - y)2

Use an Optimizer with initial parameter values of x = 0:2 and y = 10. What do
they converge to?


In [ ]:
import torch

def f(args):
  x,y = args
  return torch.exp(torch.sin(x)**2) / (x - y)**2 + (x - y)**2

param = torch.nn.Parameter(torch.tensor([0.2, 10.0]), requires_grad=True)
eta = 0.1
optimizer = torch.optim.SGD([param], lr=eta)

for epoch in range(60):
  optimizer.zero_grad()
  loss = f(param)
  loss.backward()
  optimizer.step()

print(param.data)
